# Iniciando com Word2Vec com Gensim!
  
A ideia por trás do Word2Vec é bem simples. Estamos supondo que você pode inferir o significado de uma palavra pela composição que ela mantém. Isso é análogo ao ditado popular "mostre-me seus amigos e eu direi quem você é". Então, se você tem duas palavras que têm vizinhos muito semelhantes (ou seja, o contexto de uso é aproximadamente o mesmo), então essas palavras são provavelmente muito semelhantes em significado ou pelo menos altamente relacionadas. Por exemplo, as palavras em inglês `shocked`,` appalled` e `astonished` são tipicamente usadas em um contexto similar.

A implementação do Gensim é baseada no artigo [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) e foi extendido com funcionalidades novas.

### Imports e logging

Primeiro, começamos com os imports e o logging:

In [1]:
# imports necessários para configurar o logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Em seguida, manipularemos nosso conjunto de dados. O segredo para fazer o Word2Vec funcionar é ter muitos dados no formato de texto. Neste caso, uma boa base de dados pode ser encontrada no seguinte link: [OpinRank] (http://kavita-ganesan.com/entity-ranking-data/). Este conjunto de dados tem avaliações completas de usuários de carros e hotéis. Eu concatenei especificamente todas as avaliações de hotéis em um arquivo grande que é de cerca de 97MB comprimido e 229MB não compactado. Vamos usar o arquivo compactado. Cada linha neste arquivo representa uma resenha do hotel. Você pode fazer o download do conjunto de dados do Word2Vec do OpinRank aqui.

Agora, vamos dar uma olhada mais de perto nesses dados, imprimindo a primeira linha. Você pode ver que esta é uma revisão bastante robusto.

In [7]:
data_file = "../data/reviews_data.txt.gz"

with gzip.open ('../data/reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Ler os arquivos dentro de listas
Agora que tivemos uma ideia do nosso conjunto de dados, podemos lê-lo em uma lista para que possamos passar isso para o modelo Word2Vec. Observe no código abaixo, que estou lendo diretamente o arquivo compactado. Também estamos fazendo um pré-processamento moderado das revisões usando `gensim.utils.simple_preprocess (line)`. Isso faz um pré-processamento básico, como tokenização, letras minúsculas, etc, e retorna uma lista de tokens (palavras). 



In [8]:
def read_input(input_file):
    """Esse método lê o arquivo de entrada no formato gzip"""
    
    logging.info("lendo arquivo {0}...esse processo pode demorar".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("lido {0} revisoes".format (i))
            # feito alguns pre-processamentos e retorna uma lista de palavras
            # para cada revisão no formato texto
            yield gensim.utils.simple_preprocess (line)

# ler uma revisão tokenizada em uma lista
# cada revisão se torna um série de palavras 
# então isso se torna uma lista de listas
documents = list (read_input (data_file))
logging.info ("Terminado de ler os arquivos")    

2019-09-11 15:26:16,810 : INFO : lendo arquivo ../data/reviews_data.txt.gz...esse processo pode demorar
2019-09-11 15:26:16,812 : INFO : lido 0 revisoes
2019-09-11 15:26:18,580 : INFO : lido 10000 revisoes
2019-09-11 15:26:20,085 : INFO : lido 20000 revisoes
2019-09-11 15:26:21,789 : INFO : lido 30000 revisoes
2019-09-11 15:26:23,446 : INFO : lido 40000 revisoes
2019-09-11 15:26:25,204 : INFO : lido 50000 revisoes
2019-09-11 15:26:27,363 : INFO : lido 60000 revisoes
2019-09-11 15:26:28,979 : INFO : lido 70000 revisoes
2019-09-11 15:26:30,683 : INFO : lido 80000 revisoes
2019-09-11 15:26:32,540 : INFO : lido 90000 revisoes
2019-09-11 15:26:34,084 : INFO : lido 100000 revisoes
2019-09-11 15:26:35,938 : INFO : lido 110000 revisoes
2019-09-11 15:26:37,289 : INFO : lido 120000 revisoes
2019-09-11 15:26:38,634 : INFO : lido 130000 revisoes
2019-09-11 15:26:40,343 : INFO : lido 140000 revisoes
2019-09-11 15:26:42,207 : INFO : lido 150000 revisoes
2019-09-11 15:26:44,116 : INFO : lido 160000 r

## Treinamento do modelo Word2Vec

Treinar o modelo é bastante simples. Você apenas instancia o Word2Vec e passa os comentários que lemos na etapa anterior (os `documentos`). Então, estamos essencialmente passando uma lista de listas. Onde cada lista na lista principal contém um conjunto de tokens de uma revisão do usuário. O Word2Vec usa todos esses tokens para criar internamente um vocabulário. E por vocabulário, quero dizer um conjunto de palavras únicas.

Depois de construir o vocabulário, só precisamos chamar `train(...)` para começar a treinar o modelo Word2Vec. O treinamento no conjunto de dados [OpinRank] (http://kavita-ganesan.com/entity-ranking-data/) leva cerca de 10 minutos, portanto, seja paciente ao executar seu código neste conjunto de dados.

Nos bastidores, na verdade, estamos treinando uma rede neural simples com uma única camada oculta. Mas, na verdade, não vamos usar a rede neural após o treinamento. Em vez disso, o objetivo é aprender os pesos da camada oculta. Esses pesos são essencialmente os vetores de palavras que estamos tentando aprender.

In [9]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-09-11 15:26:59,302 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-09-11 15:26:59,304 : INFO : collecting all words and their counts
2019-09-11 15:26:59,305 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-11 15:26:59,532 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-09-11 15:26:59,751 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-09-11 15:27:00,008 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-09-11 15:27:00,251 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-09-11 15:27:00,520 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-09-11 15:27:00,783 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2019-09-11 15:27:01,004 : INFO : PROGRESS: a

2019-09-11 15:27:37,214 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-09-11 15:27:37,216 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-09-11 15:27:37,218 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-09-11 15:27:37,219 : INFO : EPOCH - 1 : training on 41519358 raw words (30349153 effective words) took 31.0s, 978295 effective words/s
2019-09-11 15:27:38,247 : INFO : EPOCH 2 - PROGRESS: at 2.79% examples, 853770 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:27:39,248 : INFO : EPOCH 2 - PROGRESS: at 5.75% examples, 876962 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:27:40,249 : INFO : EPOCH 2 - PROGRESS: at 8.89% examples, 915126 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:27:41,260 : INFO : EPOCH 2 - PROGRESS: at 11.42% examples, 917371 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:27:42,264 : INFO : EPOCH 2 - PROGRESS: at 14.05% examples, 921589 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:27:43,

2019-09-11 15:28:39,893 : INFO : EPOCH 3 - PROGRESS: at 86.89% examples, 819351 words/s, in_qsize 17, out_qsize 2
2019-09-11 15:28:40,898 : INFO : EPOCH 3 - PROGRESS: at 90.04% examples, 821057 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:28:41,910 : INFO : EPOCH 3 - PROGRESS: at 93.24% examples, 824392 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:28:42,926 : INFO : EPOCH 3 - PROGRESS: at 96.48% examples, 827059 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:28:43,928 : INFO : EPOCH 3 - PROGRESS: at 99.33% examples, 826843 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:28:44,062 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-09-11 15:28:44,075 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-09-11 15:28:44,087 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-09-11 15:28:44,096 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-09-11 15:28:44,102 : INFO : worker thread finished; awaiting 

2019-09-11 15:29:35,546 : INFO : EPOCH 5 - PROGRESS: at 63.55% examples, 970482 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:29:36,559 : INFO : EPOCH 5 - PROGRESS: at 66.94% examples, 970783 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:29:37,588 : INFO : EPOCH 5 - PROGRESS: at 70.23% examples, 970857 words/s, in_qsize 16, out_qsize 3
2019-09-11 15:29:38,591 : INFO : EPOCH 5 - PROGRESS: at 73.72% examples, 972109 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:29:39,598 : INFO : EPOCH 5 - PROGRESS: at 76.89% examples, 973055 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:29:40,601 : INFO : EPOCH 5 - PROGRESS: at 80.04% examples, 973432 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:29:41,608 : INFO : EPOCH 5 - PROGRESS: at 83.29% examples, 973133 words/s, in_qsize 17, out_qsize 2
2019-09-11 15:29:42,610 : INFO : EPOCH 5 - PROGRESS: at 86.46% examples, 973202 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:29:43,610 : INFO : EPOCH 5 - PROGRESS: at 90.05% examples, 974014 words/s,

2019-09-11 15:30:28,718 : INFO : EPOCH 2 - PROGRESS: at 42.28% examples, 1108503 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:30:29,723 : INFO : EPOCH 2 - PROGRESS: at 46.35% examples, 1109981 words/s, in_qsize 20, out_qsize 0
2019-09-11 15:30:30,727 : INFO : EPOCH 2 - PROGRESS: at 50.15% examples, 1108579 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:30:31,728 : INFO : EPOCH 2 - PROGRESS: at 53.75% examples, 1107968 words/s, in_qsize 20, out_qsize 2
2019-09-11 15:30:32,729 : INFO : EPOCH 2 - PROGRESS: at 57.72% examples, 1109157 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:30:33,742 : INFO : EPOCH 2 - PROGRESS: at 61.59% examples, 1109500 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:30:34,748 : INFO : EPOCH 2 - PROGRESS: at 65.61% examples, 1109928 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:30:35,768 : INFO : EPOCH 2 - PROGRESS: at 69.34% examples, 1109320 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:30:36,769 : INFO : EPOCH 2 - PROGRESS: at 72.96% examples, 1108271

2019-09-11 15:31:22,533 : INFO : EPOCH 4 - PROGRESS: at 38.36% examples, 1112769 words/s, in_qsize 20, out_qsize 3
2019-09-11 15:31:23,533 : INFO : EPOCH 4 - PROGRESS: at 42.47% examples, 1114906 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:31:24,542 : INFO : EPOCH 4 - PROGRESS: at 46.50% examples, 1113939 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:31:25,542 : INFO : EPOCH 4 - PROGRESS: at 50.32% examples, 1112966 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:31:26,548 : INFO : EPOCH 4 - PROGRESS: at 54.02% examples, 1113621 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:31:27,551 : INFO : EPOCH 4 - PROGRESS: at 57.94% examples, 1113451 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:31:28,562 : INFO : EPOCH 4 - PROGRESS: at 61.63% examples, 1110721 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:31:29,565 : INFO : EPOCH 4 - PROGRESS: at 65.46% examples, 1107760 words/s, in_qsize 20, out_qsize 0
2019-09-11 15:31:30,582 : INFO : EPOCH 4 - PROGRESS: at 69.20% examples, 1107424

2019-09-11 15:32:17,506 : INFO : EPOCH 6 - PROGRESS: at 37.40% examples, 1088821 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:32:18,516 : INFO : EPOCH 6 - PROGRESS: at 41.33% examples, 1087263 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:32:19,525 : INFO : EPOCH 6 - PROGRESS: at 45.39% examples, 1088136 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:32:20,529 : INFO : EPOCH 6 - PROGRESS: at 49.31% examples, 1090873 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:32:21,530 : INFO : EPOCH 6 - PROGRESS: at 52.92% examples, 1090905 words/s, in_qsize 19, out_qsize 1
2019-09-11 15:32:22,537 : INFO : EPOCH 6 - PROGRESS: at 56.83% examples, 1092238 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:32:23,539 : INFO : EPOCH 6 - PROGRESS: at 60.61% examples, 1092541 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:32:24,541 : INFO : EPOCH 6 - PROGRESS: at 64.61% examples, 1092784 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:32:25,550 : INFO : EPOCH 6 - PROGRESS: at 68.34% examples, 1094033

2019-09-11 15:33:11,602 : INFO : EPOCH 8 - PROGRESS: at 33.86% examples, 1095432 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:33:12,607 : INFO : EPOCH 8 - PROGRESS: at 37.74% examples, 1095728 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:33:13,615 : INFO : EPOCH 8 - PROGRESS: at 41.74% examples, 1094407 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:33:14,621 : INFO : EPOCH 8 - PROGRESS: at 45.70% examples, 1094457 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:33:15,630 : INFO : EPOCH 8 - PROGRESS: at 49.55% examples, 1094831 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:33:16,641 : INFO : EPOCH 8 - PROGRESS: at 53.22% examples, 1095311 words/s, in_qsize 17, out_qsize 2
2019-09-11 15:33:17,652 : INFO : EPOCH 8 - PROGRESS: at 57.15% examples, 1095743 words/s, in_qsize 20, out_qsize 0
2019-09-11 15:33:18,658 : INFO : EPOCH 8 - PROGRESS: at 60.88% examples, 1094749 words/s, in_qsize 20, out_qsize 0
2019-09-11 15:33:19,661 : INFO : EPOCH 8 - PROGRESS: at 64.86% examples, 1094620

2019-09-11 15:34:05,676 : INFO : EPOCH 10 - PROGRESS: at 25.87% examples, 972801 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:34:06,733 : INFO : EPOCH 10 - PROGRESS: at 28.41% examples, 938351 words/s, in_qsize 17, out_qsize 2
2019-09-11 15:34:07,746 : INFO : EPOCH 10 - PROGRESS: at 30.70% examples, 911266 words/s, in_qsize 20, out_qsize 0
2019-09-11 15:34:08,753 : INFO : EPOCH 10 - PROGRESS: at 33.62% examples, 902758 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:34:09,754 : INFO : EPOCH 10 - PROGRESS: at 37.40% examples, 917571 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:34:10,756 : INFO : EPOCH 10 - PROGRESS: at 41.48% examples, 931436 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:34:11,771 : INFO : EPOCH 10 - PROGRESS: at 44.82% examples, 929444 words/s, in_qsize 18, out_qsize 1
2019-09-11 15:34:12,772 : INFO : EPOCH 10 - PROGRESS: at 48.63% examples, 940115 words/s, in_qsize 19, out_qsize 0
2019-09-11 15:34:13,786 : INFO : EPOCH 10 - PROGRESS: at 52.36% examples, 948237

(303485451, 415193580)

## Vamos checar o output 
Este primeiro exemplo mostra um caso simples de procurar palavras semelhantes à palavra `dirty`. Tudo o que precisamos fazer aqui é chamar a função `most_similar` e fornecer a palavra` dirty` como o exemplo positivo. Isso retorna o top 10 palavras semelhantes.

In [10]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

2019-09-11 15:34:28,504 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8593920469284058),
 ('stained', 0.7805181741714478),
 ('smelly', 0.7688528299331665),
 ('dusty', 0.7665583491325378),
 ('unclean', 0.763653576374054),
 ('grubby', 0.7523776292800903),
 ('gross', 0.7272483110427856),
 ('soiled', 0.7140095233917236),
 ('disgusting', 0.7074264287948608),
 ('dingy', 0.7069816589355469)]

Isso parece muito bom, certo? Vamos dar uma olhada em mais alguns. Vamos ver a similaridade de `educado`,` frança` e `chocado`. 

In [11]:
# vamos ver as 6 palavras mais similares a 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1, topn=6)

[('courteous', 0.917888343334198),
 ('friendly', 0.8240551948547363),
 ('cordial', 0.796963095664978),
 ('professional', 0.7924618721008301),
 ('attentive', 0.775162935256958),
 ('curteous', 0.7709380984306335)]

In [12]:
# vamos ver as 6 palavras mais similares a 'france'
w1 = ["france"]
model.wv.most_similar(positive=w1, topn=6)

[('canada', 0.6640556454658508),
 ('germany', 0.6611374616622925),
 ('spain', 0.6501389741897583),
 ('england', 0.6393054723739624),
 ('mexico', 0.5927436351776123),
 ('thailand', 0.5918599367141724)]

In [13]:
# vamos ver as 6 palavras mais similares a 'france'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)

[('horrified', 0.8404176235198975),
 ('astonished', 0.7953061461448669),
 ('amazed', 0.7878398895263672),
 ('appalled', 0.7607062458992004),
 ('dismayed', 0.7574317455291748),
 ('stunned', 0.7563004493713379)]

Isso é bom. Você pode até especificar vários exemplos positivos para obter coisas que estão relacionadas no contexto fornecido e fornecer exemplos negativos para dizer o que não deve ser considerado como relacionado. No exemplo abaixo, estamos pedindo todos os itens que se referem *cama* (*bed* em inglês):

In [14]:
# palavras relacionadas com cama
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

[('duvet', 0.70943284034729),
 ('blanket', 0.6876029968261719),
 ('mattress', 0.6864297389984131),
 ('matress', 0.6831876039505005),
 ('quilt', 0.676246702671051),
 ('pillowcase', 0.6585795879364014),
 ('sheets', 0.636309027671814),
 ('pillows', 0.6242099404335022),
 ('foam', 0.6231487989425659),
 ('pillowcases', 0.6110523343086243)]

### Similaridade entre duas palavras no vocabulário

Você pode até usar o modelo Word2Vec para retornar a semelhança entre duas palavras que estão presentes no vocabulário.

In [15]:
# similaridade de duas palavras diferentes
model.wv.similarity(w1="dirty", w2="smelly")

0.76885283

In [16]:
# similaridades de duas palavras idênticas
model.wv.similarity(w1="dirty", w2="dirty")

1.0

In [17]:
# similaridade de duas palavras opostas
model.wv.similarity(w1="dirty", w2="clean")

0.26405865

Debaixo dos panos, os três trechos acima calculam a semelhança de cosseno entre as duas palavras especificadas usando vetores de palavras de cada um. A partir das pontuações, faz sentido que `dirty` seja altamente similar a` smelly`, mas `dirty` é diferente de` clean`. Se você fizer uma semelhança entre duas palavras idênticas, a pontuação será 1.0, já que o intervalo da pontuação de semelhança do cosseno será sempre entre [0.0-1.0]. Você pode ler mais sobre a pontuação de semelhança de cosseno [aqui] (https://en.wikipedia.org/wiki/Cosine_similarity).

### Encontre o estranho
Você pode até usar o Word2Vec para encontrar itens estranhos com uma lista de itens.

In [18]:
# Qual dos items abaixo é o estranho da lista?
model.wv.doesnt_match(["cat","dog","france"])

/home/geanderson/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [19]:
# Qual dos items abaixo é o estranho da lista?
model.wv.doesnt_match(["bed","pillow","duvet","car"])

'car'

## Entendendo os parâmetros que podem ser utilizados
Para treinar o modelo anteriormente, tivemos que definir alguns parâmetros. Agora, vamos tentar entender o que alguns deles significam. Para referência, este é o comando que usamos para treinar o modelo.

```
model = gensim.models.Word2Vec(documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
O tamanho do vetor denso para representar cada token ou palavra. Se você tiver dados muito limitados, o tamanho deverá ser um valor muito menor. Se você tiver muitos dados, é bom experimentar vários tamanhos. Um valor de 100-150 funcionou bem para o dataset treinado.

### `window`
A distância máxima entre a palavra alvo e a palavra vizinha. Se a posição do seu vizinho for maior que a largura máxima da janela à esquerda e à direita, alguns vizinhos não serão considerados como relacionados à palavra de destino. Em teoria, uma janela menor deve fornecer termos mais relacionados. Se você tiver muitos dados, o tamanho da janela não deve importar muito, desde que seja uma janela de tamanho decente. 

### `min_count`
Contagem de frequência mínima de palavras. O modelo ignoraria as palavras que não statisfy o `min_count`. Palavras extremamente raras geralmente não são importantes, então é melhor livrar-se delas. A menos que seu conjunto de dados seja realmente pequeno, isso não afeta realmente o modelo.

### `workers`
Quantas threads deveríamos usar?

## Quando usar Word2Vec?

Existem muitos cenários de aplicativos para o Word2Vec. Imagine se você precisa construir um léxico de sentimento. Treinar um modelo Word2Vec em grandes quantidades de comentários de usuários ajuda você a conseguir isso. Você tem um léxico não apenas para o sentimento, mas para a maioria das palavras no vocabulário.

Além dos dados de texto bruto não estruturados, você também pode usar o Word2Vec para obter dados mais estruturados. Por exemplo, se você tivesse tags para um milhão de perguntas e respostas do stackoverflow, poderia encontrar tags relacionadas a uma determinada tag e recomendar as relacionadas para exploração. Você pode fazer isso tratando cada conjunto de tags de coexistência como uma "frase" e treinar um modelo Word2Vec nesses dados. Concedido, você ainda precisa de um grande número de exemplos para fazê-lo funcionar.